In [ ]:
import os
import sys

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

import pandas as pd
import math
import datetime
import time
from eth_utils import to_bytes, to_int
import requests
from pprint import pprint

from src.clients.aave_client import AaveClient


In [ ]:
# Setup client
RPC = "ARBITRUM"  # or POLYGON, ETHEREUM, KOVAN, MUMBAI
RPC_URL = os.getenv(f"{RPC}_RPC_URL")

In [ ]:
# Initialize Client:
aave_client = AaveClient(wallet_address=os.getenv('WALLET_ADDRESS'),
                         private_key=os.getenv('PRIVATE_WALLET_KEY'),
                         network=RPC,
                         rpc_url=RPC_URL,
                         gas_strategy="medium")

# Get the lending pool smart contract:
lending_pool = aave_client.get_lending_pool()
print(lending_pool.address)

In [ ]:
# Wallet data
available_borrow_base, total_debt_base, total_collateral_base = aave_client.get_user_data(lending_pool)
print(f"Available borrow: {available_borrow_base}, Total debt: {total_debt_base}, Total collateral: {total_collateral_base}")

In [ ]:
# Call protocol functions
assets = aave_client.get_protocol_data("getAllReservesTokens")
print("All reserve tokens:", assets)

asset_address = '0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9'
user_address = os.getenv('WALLET_ADDRESS')

user_reserves = aave_client.get_protocol_data("getUserReserveData", asset_address, user_address)
print("User reserve data:", user_reserves)

In [ ]:
# Call pool functions
assets_symbols = ['DAI', 'USDC', 'USDT']
filtered_assets = [asset for asset in assets if asset[0] in assets_symbols]

df = pd.DataFrame(columns=['symbol', 'currentLiquidityRate', 'currentVariableBorrowRate', 'lastUpdateTimestamp'])

for symbol, asset_address in filtered_assets:
    reserve_data = aave_client.get_pool_data(lending_pool, "getReserveData", asset_address)
    current_liquidity_rate = reserve_data['currentLiquidityRate'] * 100
    current_variable_borrow_rate = reserve_data['currentVariableBorrowRate'] * 100
    last_update_timestamp = pd.to_datetime(reserve_data['lastUpdateTimestamp'], unit='s')
    
    df.loc[asset_address] = [symbol, current_liquidity_rate, current_variable_borrow_rate, last_update_timestamp]

print(df)

In [ ]:
# Calculate spread
spread = df['currentLiquidityRate'].max() - df['currentVariableBorrowRate'].min()
buy_asset = df['currentLiquidityRate'].idxmax()
sell_asset = df['currentVariableBorrowRate'].idxmin()
print(f"Spread: {spread}, Buy asset: {buy_asset}, Sell asset: {sell_asset}")

In [ ]:
# Calculate leverage
LTV = 0.9
initial_collateral = 100
stop_condition = 0.9

number_of_loops = math.ceil(math.log(stop_condition) / math.log(LTV))
total_collateral = initial_collateral * ((1 - LTV**(number_of_loops + 1)) / (1 - LTV))
leverage = total_collateral / initial_collateral

print(f"Number of loops: {number_of_loops}, Total collateral: {total_collateral}, Leverage: {leverage}")


In [ ]:
# Calculate APY
apy = (df['currentLiquidityRate'].max() * initial_collateral + (total_collateral - initial_collateral) * max(spread, 0)) / initial_collateral
print(f"APY: {apy}")

In [ ]:
# Get user account data
user_account_data = aave_client.get_pool_data(lending_pool, "getUserAccountData", user_address)
print("User account data:", user_account_data)

In [ ]:
# Get user configuration
user_reserve_data = aave_client.get_pool_data(lending_pool, "getUserConfiguration", user_address)
print("User reserve data:", user_reserve_data)

In [ ]:

# Interpret user reserve data
def interpret_user_reserve_data(user_reserve_data):
    user_reserve_value = user_reserve_data['data']
    binary_data = bin(user_reserve_value)[2:].zfill(256)
    bit_pairs = [binary_data[i:i+2] for i in range(0, len(binary_data), 2)]

    borrowed_indices = []
    collateral_indices = []

    for i in range(len(bit_pairs)-1, -1, -1):
        pair = bit_pairs[i]
        asset_index = len(bit_pairs) - i - 1

        if pair == "01":
            borrowed_indices.append(asset_index)
        elif pair == "11":
            collateral_indices.append(asset_index)
            borrowed_indices.append(asset_index)
        elif pair == "10":
            collateral_indices.append(asset_index)

    return borrowed_indices, collateral_indices

borrowed_indices, collateral_indices = interpret_user_reserve_data(user_reserve_data)

reserves_assets_list = aave_client.get_pool_data(lending_pool, "getReservesList")

print("Supplied asset is: ", reserves_assets_list[collateral_indices[0]])
print("Borrowed asset is: ", reserves_assets_list[borrowed_indices[0]])

In [ ]:
supply_asset = reserves_assets_list[collateral_indices[0]]
supply_reserve_asset = next((token for token in aave_client.active_network.aave_tokens if token.address == supply_asset), None)
supply_reserves = aave_client.get_protocol_data("getUserReserveData", supply_asset, user_address)

borrow_asset = reserves_assets_list[borrowed_indices[0]]
borrow_reserve_asset = next((token for token in aave_client.active_network.aave_tokens if token.address == borrow_asset), None)
borrow_reserves = aave_client.get_protocol_data("getUserReserveData", borrow_asset, user_address)

print(f"Supplied asset address is {supply_asset} for a total of {supply_reserves[0]}")
print(f"Borrowed asset address is {borrow_asset} for a total of {borrow_reserves[2]}")

In [ ]:

# Check wallet balances
borrow_asset_wallet_balance = aave_client.get_wallet_balance_data("balanceOf", user_address, borrow_asset)
buy_asset_balance = aave_client.get_wallet_balance_data("balanceOf", user_address, buy_asset)

print(f"Wallet has {aave_client.convert_from_decimal_units(borrow_reserve_asset, borrow_asset_wallet_balance)} balance of the borrowed assets")
print(f"Wallet has {aave_client.convert_from_decimal_units(borrow_reserve_asset, buy_asset_balance)} balance of the asset to be supplied")


In [ ]:
# Example operations (uncomment to execute)

# Deposit tokens
# deposit_token = buy_asset
# reserve_token = next((token for token in aave_client.active_network.aave_tokens if token.address == deposit_token), None)
# deposit_amount = aave_client.convert_from_decimal_units(reserve_token, buy_asset_balance)
# deposit_hash = aave_client.deposit(deposit_token=reserve_token, deposit_amount=deposit_amount)
# print("Deposit Transaction Hash:", deposit_hash)

# Withdraw tokens
# withdraw_amount = aave_client.convert_from_decimal_units(supply_reserve_asset, supply_reserves[0])
# withdraw_transaction_receipt = aave_client.withdraw(withdraw_token=supply_reserve_asset, withdraw_amount=withdraw_amount)
# print("Withdraw Transaction Hash:", withdraw_transaction_receipt)

# Swap tokens
# swap_from_token = next((token for token in aave_client.active_network.aave_tokens if token.address == supply_asset), None)
# swap_to_token = next((token for token in aave_client.active_network.aave_tokens if token.address == buy_asset), None)
# amount_to_swap = withdraw_amount
# try:
#     receipt = aave_client.swap(swap_from_token=swap_from_token, swap_to_token=swap_to_token, amount_to_swap=amount_to_swap)
#     print(f"Swap transaction completed. Receipt: {receipt}")
# except Exception as e:
#     print(f"Error executing swap: {e}")

# Repay tokens
# repay_amount = aave_client.convert_from_decimal_units(borrow_reserve_asset, borrow_asset_wallet_balance)
# repay_transaction_receipt = aave_client.repay(lending_pool_contract=lending_pool, repay_amount=repay_amount, repay_asset=borrow_reserve_asset)
# print("Repay Transaction Hash:", repay_transaction_receipt)

# Borrow tokens
# borrow_amount = LTV * aave_client.convert_from_decimal_units(supply_reserve_asset, supply_reserves[0])
# borrow_transaction_receipt = aave_client.borrow(lending_pool_contract=lending_pool, borrow_amount=borrow_amount, borrow_asset=borrow_reserve_asset)
# print("Borrow Transaction Hash:", borrow_transaction_receipt)